In [ ]:
import numpy as np
import pandas as pd
import random


# Paramètres
num_positions = 181
num_time_steps = 5

max_speed = 10.0  # Vitesse maximale

all_examples = pd.DataFrame()

for example_idx in range(100000):
    doa_indices = [random.randint(30,150), random.randint(30,150)]
    # Générer une série temporelle de 5 mesures associée à une vitesse
    time_series_example = generate_time_series_example(num_positions, num_time_steps, doa_indices, max_speed)



    # Ajouter la ligne à l'ensemble des exemples
    all_examples = pd.concat([all_examples, time_series_example], axis=0, ignore_index=True)

# Enregistrer toutes les séries temporelles dans un seul fichier CSV
all_examples.to_csv('temporal_measures_trainset.csv', index=False)

In [ ]:
def generate_time_series_example(num_positions, num_time_steps, doa_indices, max_speed):
    # Générer une vitesse aléatoire pour chaque exemple
    initial_doa_indices = doa_indices
    speeds = []
    for i in range(len(doa_indices)):
        speed = (1,-1)[np.random.randint(0,2)==1]
        if speed == 1:
            speed*= random.randint(1, int((abs(180-doa_indices[i]))/num_time_steps))
        else:
            speed *= random.randint(1,int(doa_indices[i]/num_time_steps))
        speeds+=speed,
    # Normaliser la vitesse
    #normalized_speed = speed / np.mean(speed)

    # Initialiser une série temporelle de positions pour chaque exemple
    time_series_positions = []

    for _ in range(num_time_steps):
        # Générer un vecteur de positions initialisé à zéro
        positions = np.zeros(num_positions)

        # Placer des '1' aux indices spécifiés pour les DoA
        positions[list(doa_indices)] = 1

        # Ajouter la série temporelle de positions à la liste
        time_series_positions.append(positions)

        # Mettre à jour les indices des DoA en fonction de la vitesse
        doa_indices = np.array(doa_indices) + np.array(speeds)
    # Créer un dataframe pandas avec la série temporelle de positions et la vitesse associée

    time_series_positions= np.array(time_series_positions)
    # Créer la DataFrame
    df = pd.DataFrame({i: time_series_positions[i].reshape(1,181).tolist() for i in range(time_series_positions.shape[0])})

    speeds_res = np.zeros(num_positions)
    speeds_res[initial_doa_indices] = speeds
    df['Speed'] = speeds_res.reshape(1,181).tolist()
    return df

In [ ]:
def get_dataset(path):
  data = pd.read_csv(path,header=None, index_col=False)
  X = data.iloc[1:,:5].values
  y = data.iloc[1:,5].values
  sy = list(map(lambda yy: yy.strip('][').split(', '),y))
  y = np.array(sy).astype(np.float32)
  X = np.array([ list(map(lambda mesure: mesure.strip('][').split(', '),X[i]))for i in range(len(X))]).astype(np.float32)
  return X,y

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Générer une dataset de forme (n, 5, 181)
# Remplacez cette partie avec votre propre génération de données
X,y = get_dataset("temporal_measures_trainset.csv")
n = X.shape[0]

# Créer le modèle RNN
model = Sequential()
model.add(LSTM(181, input_shape=(5, 181), activation='linear'))
model.add(Dense(181, activation='linear'))

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error')


In [3]:
from Models.RNNmodel import RNNModel
model = RNNModel()
model.load("")

In [ ]:
# Faire des prédictions sur de nouvelles données (remplacez avec vos données de test)
Xtest,ytest = get_dataset("temporal_measures_testset.csv")
predictions = model.predict(Xtest)

print("Predictions:", predictions)